<a href="https://colab.research.google.com/github/MathewBiddle/ioos_metrics/blob/yearly_ra_gts/notebooks/IOOS_obs_2_NDBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install erddapy

In [2]:
import pandas as pd
from erddapy import ERDDAP

# Function to grab the data.

In [3]:
def get_ndbc_full_stats():
    e = ERDDAP(
        server="https://erddap.ioos.us/erddap",
        protocol="tabledap",
    )
    e.response = "csv"
    dsets = {"IOOS": "gts_regional_statistics",
             "NDBC": "gts_ndbc_statistics",
             "non-NDBC": "gts_non_ndbc_statistics"}
    df_out = pd.DataFrame()
    for key, value in dsets.items():
        e.dataset_id = value
        df = e.to_pandas(
            index_col="time (UTC)",
            parse_dates=True
        )
        df["source"] = key
        df_out = pd.concat([df_out,df])
    return df_out

# Go get the data and return the dataframe

In [4]:
df = get_ndbc_full_stats()

df['total'] = df['met'] + df['wave']

df

,Year,Month,locationID,region,sponsor,met,wave,source,total
time (UTC),,,,,,,,,
2018-01-01 00:00:00+00:00,2018,1,46108,AOOS,ALASKA OCEAN OBSERVING SYSTEM,0,2592,IOOS,2592
2018-01-01 00:00:00+00:00,2018,1,AJXA2,AOOS,MARINE EXCHANGE OF ALASKA,8796,0,IOOS,8796
2018-01-01 00:00:00+00:00,2018,1,CDXA2,AOOS,MARINE EXCHANGE OF ALASKA,4782,0,IOOS,4782
2018-01-01 00:00:00+00:00,2018,1,ERXA2,AOOS,MARINE EXCHANGE OF ALASKA,5634,0,IOOS,5634
2018-01-01 00:00:00+00:00,2018,1,GIXA2,AOOS,MARINE EXCHANGE OF ALASKA,8798,0,IOOS,8798
...,...,...,...,...,...,...,...,...,...
2025-03-01 00:00:00+00:00,2025,3,OCSM2,NaN,U.S. ARMY CORPS OF ENGINEERS,0,0,non-NDBC,0
2025-03-01 00:00:00+00:00,2025,3,44097,NaN,U.S. ARMY CORPS OF ENGINEERS,0,2976,non-NDBC,2976
2025-03-01 00:00:00+00:00,2025,3,FRFN7,NaN,U.S. ARMY CORPS OF ENGINEERS,0,0,non-NDBC,0


# Compute yearly totals of number of IOOS messages sent to the GTS.

In [5]:
df.groupby(by=["source"])[['met','wave','total']].sum()

,met,wave,total
source,,,
IOOS,82005740,11767770,93773510
NDBC,58524762,13386048,71910810
non-NDBC,371637998,19140012,390778010


In [6]:
df.index.min().strftime('%Y')

'2018'

In [10]:
first_year = df.index.min().strftime('%Y')

ioos_total = df.loc[df['source']=='IOOS','total'].sum()

ioos_pcnt = ioos_total / df['total'].sum()

total = df['total'].sum()

print(f'Since {first_year} IOOS has contributed {ioos_total} messages to the GTS, which composes {(ioos_pcnt)*100:.2f} % of the total messages ({total}) reported by NDBC to the GTS')

Since 2018 IOOS has contributed 93773510 messages to the GTS, which composes 16.85 % of the total messages (556462330) reported by NDBC to the GTS


In [11]:
yearly_totals = df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum()

yearly_totals['total'] = yearly_totals['met'] + yearly_totals['wave']

print(yearly_totals.loc['IOOS'].to_markdown(floatfmt=''))

| time (UTC)                |        met |      wave |      total |
|:--------------------------|-----------:|----------:|-----------:|
| 2018-12-31 00:00:00+00:00 | 10250188.0 | 1586648.0 | 11836836.0 |
| 2019-12-31 00:00:00+00:00 | 11344260.0 | 1682754.0 | 13027014.0 |
| 2020-12-31 00:00:00+00:00 | 10501536.0 | 1410472.0 | 11912008.0 |
| 2021-12-31 00:00:00+00:00 | 10189996.0 | 1610856.0 | 11800852.0 |
| 2022-12-31 00:00:00+00:00 | 11295426.0 | 1616542.0 | 12911968.0 |
| 2023-12-31 00:00:00+00:00 | 12719038.0 | 1796258.0 | 14515296.0 |
| 2024-12-31 00:00:00+00:00 | 12794686.0 | 1726010.0 | 14520696.0 |
| 2025-12-31 00:00:00+00:00 |  2910610.0 |  338230.0 |  3248840.0 |


In [12]:
ioos_year = yearly_totals.loc['IOOS','2024']['total'].iloc[0]

ioos_year

np.int64(14520696)

In [42]:
year = '2024'

all_year = yearly_totals.loc['NDBC',year]['total'] + yearly_totals.loc['non-NDBC',year]['total'] + yearly_totals.loc['IOOS',year]['total']

ioos_year = yearly_totals.loc['IOOS','2024']['total'].iloc[0]

pcnt_year = ioos_year / all_year.iloc[0]

platforms = len(df.loc[(df.index >= '2024-01-01') & (df.index <= '2024-12-31') & (df['source']=='IOOS')].locationID.unique())

print(f'In {year}, the IOOS regions have submitted {ioos_year} messages to the GTS (from {platforms} unique platforms). Which comprises {pcnt_year*100:.2f} % of the total messages sent to the GTS via NDBC.')

In 2024, the IOOS regions have submitted 14520696 messages to the GTS (from 229 unique platforms). Which comprises 17.20 % of the total messages sent to the GTS via NDBC.


# Compute yearly totals of IOOS messages sent to the GTS by region.

In [ ]:
yearly_region_totals = df.groupby(by=["source", "region", pd.Grouper(freq="YE")])[['met','wave']].sum()

yearly_region_totals['total'] = yearly_region_totals['met'] + yearly_region_totals['wave']

yearly_region_totals.reset_index('region',inplace=True)

print(yearly_region_totals.loc['IOOS'].to_markdown(floatfmt=''))

# Calculate the percentage of IOOS OBS per year

In [ ]:
yearly_totals = df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum()

yearly_totals['total'] = yearly_totals['met'] + yearly_totals['wave']

pcnt_obs_year = yearly_totals.loc['IOOS'] / (yearly_totals.loc['IOOS'] + yearly_totals.loc['NDBC'] + yearly_totals.loc['non-NDBC'])

# pcnt_obs_year['total'] =

print(pcnt_obs_year.to_markdown())

In [ ]:
yearly_totals.loc['IOOS'] + yearly_totals.loc['NDBC'] + yearly_totals.loc['non-NDBC']

# totals['total'] = totals['met']+totals['wave']

# totals['total'].sum()

#totals

In [ ]:
yearly_totals.loc['IOOS','total'].sum()

# Total number of unique platforms submitting data to the GTS from the IOOS regions

IOOS location IDs

In [ ]:
df.loc[df['source']=='IOOS'].locationID

In [ ]:
n_ioos_stn = len(df.loc[df['source']=='IOOS'].locationID.unique())

All location ID's

In [ ]:
n_all_stn = len(df.locationID.unique())

n_all_stn

IOOS stations represent this of the total # stations whose data are delivered by NDBC to the GTS

In [ ]:
station_pcnt = n_ioos_stn / n_all_stn

print(f'Since {first_year} IOOS regions have sent {n_ioos_stn} unique stations\' data to the GTS through NDBC. This comprises nearly {station_pcnt*100:.2f}% of all the stations reported ({n_all_stn}) and more than one third of the NOS stations')

Focusing on station platform counts (260, I believe) - Can you compute what % that is of the total delivered by NDBC to the GTS? (I didn't notice if you identify the total # stations anywhere).

So, we can say "IOOS stations represent x% of the total # stations whose data are delivered by NDBC to the GTS"

In [ ]:
ioos_ndbc_non_NDBC = yearly_totals.loc['IOOS'] + yearly_totals.loc['NDBC'] + yearly_totals.loc['non-NDBC']

yearly_totals.loc['IOOS','total'].sum() / ioos_ndbc_non_NDBC['total'].sum()

# Total number of sponsors submitting data to the GTS via IOOS Regions

In [ ]:
df.loc[df['source']=='IOOS'].sponsor

In [ ]:
len(df.loc[df['source']=='IOOS'].sponsor.unique())

# Average number of messages (met+wave) per year

For all the years, including 2025

In [ ]:
print(df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum().loc['IOOS'].mean().to_markdown(floatfmt=''))
print(df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum().loc['IOOS'].mean().sum())

for 2018-2024, since 2025 is only through March.

In [ ]:
print(df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum().loc['IOOS'].loc[:"2024"].mean().to_markdown(floatfmt=''))
print(df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum().loc['IOOS'].loc[:"2024"].mean().sum())
avg = df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum().loc['IOOS'].loc[:"2024"].mean().sum()

print(f'On average, IOOS regions submit a total of {avg:.2f} (~13 million!) messages per year to the GTS.')

## Total number of stations

In [ ]:
print(len(df.locationID.unique()))

print(df.locationID.unique())

In [ ]:
260/1039

In [ ]:
df.locationID

"IOOS contributes the largest number of NOS platforms to the GTS." For the latter, we'd need to identify "NOS" (not to confuse with an earlier comment in a different issue about defining NOS, in which case I was equating NOS with CO-OPS based on the non-NDBC report labels). In this case,
1. NOS = IOOS-regional (all) + non-NDBC (National ocean service, NOAA NOS PORTS, CBIBS, and NERRS.)

2. Perhaps add another condition where if there are 0s for stations, then omit from the station count?

3. And constrain the station count to only the Calendar Year 2024.

In [ ]:
non_ndbc_nos_list = ['CHESAPEAKE BAY INTERPRETIVE BUOY SYSTEM',
'NATIONAL ESTUARINE RESEARCH RESERVE SYSTEM',
'NATIONAL OCEAN SERVICE',
'NOAA NOS PHYSICAL OCEANOGRAPHIC RT SYSTEM PROGRAM']

df_non_ndbc = df.loc[df['source']=='non-NDBC']

nos = df_non_ndbc[df_non_ndbc['sponsor'].isin(non_ndbc_nos_list)]
print('total NOS:',len(nos))
condition = (nos['met'] ==0) & (nos['wave']==0)
print('NOS empty',len(condition.loc[condition]))
nos = nos[~condition]
print('total NOS-empty:',len(nos))


ioos = df.loc[df['source']=='IOOS']
print('total IOOS:',len(ioos))
condition = (ioos['met'] ==0) & (ioos['wave']==0)
print('IOOS empty',len(condition.loc[condition]))
ioos = ioos[~condition]
print('total IOOS-empty:',len(ioos))

ioos_platform_count = len(ioos.locationID.unique())
nos_platform_count = len(nos.locationID.unique())

print(f'{ioos_platform_count} / {nos_platform_count + ioos_platform_count} = **{(ioos_platform_count/(nos_platform_count+ioos_platform_count))*100:.2f}%**')

In [ ]:
ioos = df.loc[df['source']=='IOOS']
ioos.loc[condition]['locationID'].unique()

In [ ]:
ioos = df.loc[df['source']=='IOOS']
len(ioos.loc[(ioos['total_messages']!=0)].locationID.unique())

In [ ]:
for loc in nos.locationID.unique():
  if len(nos.loc[nos['locationID']==loc].source.unique()) >1:
    print(loc,nos.loc[nos['locationID']==loc].source.unique())

In [ ]:
count = 0
for sponsor in non_ndbc_nos_list:
  length = len(nos.loc[nos['sponsor']==sponsor].locationID.unique())
  print(f'{sponsor}: {length}')
  count = count + length

print(f'Total = {count}')
print(f'Total w/ IOOS = {count+ioos_platform_count}')
#len(nos.loc[nos['sponsor']=='CHESAPEAKE BAY INTERPRETIVE BUOY SYSTEM'].locationID.unique())

In [ ]:
non_ndbc_nos_list = ['CHESAPEAKE BAY INTERPRETIVE BUOY SYSTEM',
'NATIONAL ESTUARINE RESEARCH RESERVE SYSTEM',
'NATIONAL OCEAN SERVICE',
'NOAA NOS PHYSICAL OCEANOGRAPHIC RT SYSTEM PROGRAM']

df_non_ndbc = df.loc[df['source']=='non-NDBC']
#print(len(df_non_ndbc))


filtered = df_non_ndbc[df_non_ndbc['sponsor'].isin(non_ndbc_nos_list)]
print('total NOS:',len(filtered))

nos = pd.concat([filtered,df.loc[df['source']=='IOOS']])
print('total IOOS:',len(nos.loc[nos['source']=='IOOS']))


# drop empty met and wave rows
condition = (nos['met'] ==0) & (nos['wave']==0)

nos = nos[~condition]
print('total IOOS-empty:',len(nos.loc[nos['source']=='IOOS']))
print('total NOS-empty:',len(nos.loc[nos['source']!='IOOS']))

ioos_platform_count = len(nos.loc[nos['source']=='IOOS'].locationID.unique())
nos_platform_count = len(nos.locationID.unique())

print(f'{ioos_platform_count} / {nos_platform_count} = **{(ioos_platform_count/nos_platform_count)*100:.2f}%**')

In [ ]:
for loc in nos.locationID.unique():
  if len(nos.loc[nos['locationID']==loc].source.unique()) >1:
    print(loc,nos.loc[nos['locationID']==loc].source.unique())

In [ ]:
nos.loc[nos['sponsor']=='NATIONAL OCEAN SERVICE'].source.unique()

In [ ]:
len(nos.loc[nos['sponsor']=='NATIONAL ESTUARINE RESEARCH RESERVE SYSTEM'].locationID.unique())

In [ ]:
len(nos.loc[nos['sponsor']=='NATIONAL OCEAN SERVICE'].locationID.unique())

In [ ]:
len(nos.loc[nos['sponsor']=='NOAA NOS PHYSICAL OCEANOGRAPHIC RT SYSTEM PROGRAM'].locationID.unique())

In [ ]:
df.loc[df['source']=='non-NDBC'].sponsor.unique()

In [70]:
df_ioos = df.loc[(df['source']=='IOOS')]

condition = (df_ioos['met'] ==0) & (df_ioos['wave']==0)

len(df_ioos[~condition].locationID.unique())

237

In [81]:
df.locationID.unique()

condition = (df['met'] ==0) & (df['wave']==0)

zero_obs = df[condition]

no_obs_loc = []

for loc in zero_obs['locationID'].unique():
  if df.loc[(df['locationID'] == loc),'total'].any() == 0:
    no_obs_loc.append(loc)
    print(f'{loc} has no obs!')

print(f'This many platforms do not have observations for the entire record: {len(no_obs_loc)}')

41058 has no obs!
IMGP4 has no obs!
AUDP4 has no obs!
FRDP4 has no obs!
VQSP4 has no obs!
YABP4 has no obs!
CQUC1 has no obs!
HBXC1 has no obs!
MYXC1 has no obs!
TDPC1 has no obs!
MBXC1 has no obs!
BDXC1 has no obs!
FPXC1 has no obs!
GRBL1 has no obs!
42046 has no obs!
LYBT2 has no obs!
45179 has no obs!
DMNO3 has no obs!
46119 has no obs!
51203 has no obs!
51204 has no obs!
46124 has no obs!
PTAC1 has no obs!
MBPA1 has no obs!
42369 has no obs!
42375 has no obs!
42392 has no obs!
42887 has no obs!
SACV4 has no obs!
VERV4 has no obs!
41116 has no obs!
44093 has no obs!
LBRF1 has no obs!
MUKF1 has no obs!
42087 has no obs!
42088 has no obs!
42089 has no obs!
42090 has no obs!
BKTL1 has no obs!
CARL1 has no obs!
CPVM2 has no obs!
DMSF1 has no obs!
FRXM3 has no obs!
GCVF1 has no obs!
JXUF1 has no obs!
LTJF1 has no obs!
MCYF1 has no obs!
OBXC1 has no obs!
PMOA2 has no obs!
PNLM6 has no obs!
PRUR1 has no obs!
PTOA1 has no obs!
PXSC1 has no obs!
RCYF1 has no obs!
ULAM6 has no obs!
WBYA1 has 

In [78]:
df.loc[(df['locationID'] == 'TDPC1') & (df['total'] = 0)]

,Year,Month,locationID,region,sponsor,met,wave,source,total
time (UTC),,,,,,,,,
2018-01-01 00:00:00+00:00,2018,1,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
2018-02-01 00:00:00+00:00,2018,2,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
2018-03-01 00:00:00+00:00,2018,3,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
2018-04-01 00:00:00+00:00,2018,4,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
2018-05-01 00:00:00+00:00,2018,5,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
...,...,...,...,...,...,...,...,...,...
2024-11-01 00:00:00+00:00,2024,11,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
2024-12-01 00:00:00+00:00,2024,12,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0
2025-01-01 00:00:00+00:00,2025,1,TDPC1,CeNCOOS,CENTRAL AND NORTHERN CALIFORNIA OCEAN OBSERVIN...,0,0,IOOS,0


To confirm that number, here we search for each of those platform IDs and calculate some statistics. If the statistics are all zero for `met`, `wave`, and `total`, then we don't have observations for all those platforms.

In [68]:
len(df.loc[(df['locationID'].isin(no_obs_loc)) & (df.index>='2024-01-01') & (df.index<='2024-12-31') & (df['source'] == 'IOOS')].locationID.unique())

15

In [69]:
260-15

245

In [ ]:
(237/975) * 100

In [ ]:
df.to_csv('gts_metrics.csv', index=False)

# From Gemini

Prompt:
1. ```Can you write a few statements about the importance of the data sourced from IOOS in relation to the entire dataset?```
2. ``` What about framing it around the total number of messages sent to the GTS by each source? ```


In [ ]:
# Create the 'total_messages' column
df['total_messages'] = df['met'] + df['wave']

# Group by source and sum 'total_messages'
messages_by_source = df.groupby('source')['total_messages'].sum().reset_index()

# Calculate the grand total of messages
grand_total_messages = messages_by_source['total_messages'].sum()

# Calculate the percentage for each source
messages_by_source['percentage'] = (messages_by_source['total_messages'] / grand_total_messages) * 100

# Sort by total messages descending for better presentation
messages_by_source = messages_by_source.sort_values(by='total_messages', ascending=False)

# Print the results
print("Total Messages Sent to GTS (met + wave) by Source:")
print(messages_by_source.to_markdown(index=False, numalign="left", stralign="left", floatfmt=".2f"))

# Store the IOOS percentage for easy reference in the text
ioos_percentage = messages_by_source.loc[messages_by_source['source'] == 'IOOS', 'percentage'].iloc[0]
ioos_total_messages = messages_by_source.loc[messages_by_source['source'] == 'IOOS', 'total_messages'].iloc[0]
non_ndbc_total_messages = messages_by_source.loc[messages_by_source['source'] == 'non-NDBC', 'total_messages'].iloc[0]

3. ```Can you also frame it around source and platformID?```

In [ ]:
# Assuming 'platformID' refers to the 'locationID' column as 'platformID' does not exist in the data.
# Group by source and count unique locationIDs
locations_per_source = df.groupby('source')['locationID'].nunique().reset_index()
locations_per_source = locations_per_source.rename(columns={'locationID': 'unique_locations'})

# Merge this with the previously calculated messages_by_source DataFrame
# (Reload messages_by_source calculation for robustness in case kernel state changed)
df['total_messages'] = df['met'] + df['wave']
messages_by_source = df.groupby('source')['total_messages'].sum().reset_index()
source_summary = pd.merge(messages_by_source, locations_per_source, on='source')

# Calculate average messages per location for each source
source_summary['avg_messages_per_location'] = source_summary['total_messages'] / source_summary['unique_locations']

# Sort by total messages descending
source_summary = source_summary.sort_values(by='total_messages', ascending=False)

# Print the summary table
print("Summary by Source (including LocationID count and Avg Messages):")
# Displaying integer values for messages for clarity
print(source_summary.to_markdown(index=False, numalign="left", stralign="left", floatfmt=",.0f"))